<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulaci-n-2/blob/main/linea_de_espera_dos_servidores_paralelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LINEAS DE ESPERA CON DOS SERVIDORES EN PARALELO

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import random as rnd

In [ ]:
def servicio(t):
  if 0 <=t<=3:
    return 5+5*t
  elif 3<t<=5:
    return 20
  elif 5<t<=9:
    return 20-2*(t-5)
  else:
    return 0

In [ ]:
def llegada(t):
  Lambda=0.34
  T=60

  llegadas=[]
  tiempos=[]

  while t <= T:
      U1=rnd.random()
      t=t-np.log(U1) / Lambda
      U2=rnd.random()

      if U2 <= servicio(t) / Lambda:
          TS=t
          llegadas.append(TS)
          tiempos.append(TS)
  return np.mean(tiempos)

## COLA DE DOS SERVIDORES EN PARALELO

Lo importante de este modelo es que al tener una sola cola que da el servicio de puestosm hay que llevar un control, identificando los elemtnos que están y el orden en el que se encuentran en cada instante de actualización.

*Variables:*
* $\;\;\;t\;\;\;$ :     Tiempo
* $N_{LL}$ : Número de llegadas hasat el instante $t$
* $C_1$: Número de clientes servidos por $S_1$ hasta $t$
* $C_2$: Número de clientes servidos por $S_2$ hasta $t$

* $n\;\;$ : Número de clientes en el sistema en el intante $t$
* $i_1$: Cliente atendido por $S_1$ en el instante $t$
* $i_2$: Cliente atendido por $S_2$ en el instante $t$
* $i_3,i_4$ si hay $n-2$ clientes ordenados en la cola en el instante $t$

In [ ]:
t=0
C1=0
C2=0
N_LL=0
SS=[0]
LAMBDA_1=0.34
LAMBDA_2=0.34
t_LL=0
t1=float('inf')
t2=float('inf')

# Funciones de llegada
def generar_tiempo_llegada():
    return random.expovariate(1.0)

# Funciones de servicio
def generar_tiempo_servicio():
    return random.expovariate(1.0)

T0=t-np.log(random.random()) / LAMBDA_1
t_LL=T0

LL = []
S = []

# Simulación
while C1 + C2 < 100:
    if t_LL <= t1 and t_LL <= t2 and len(SS) == 1 and SS[0] == 0:
        # CASO 1
        t=t_LL
        N_LL+=1
        LL.append(t)

        T_t=-np.log(random.random()) / LAMBDA_1
        t_LL=t+T_t

        if SS == [0]:  # Si ambos servidores están libres
            SS=[1, N_LL, 0]
            Y1=-np.log(random.random()) / LAMBDA_2
            t1=t+Y1
        elif SS == [1, 0]:  # Si el servidor 1 está ocupado
            SS=[2, N_LL]
            Y2=-np.log(random.random()) / LAMBDA_2
            t2=t+Y2
        elif SS == [1, N_LL, 0]:  # Si el servidor 2 está libre
            SS=[2, N_LL, N_LL]
            Y1=-np.log(random.random()) / LAMBDA_2
            t1=t+Y1
        else:
            SS.append(N_LL)

    elif t_LL > t1 and t1 <= t2 and len(SS) > 1:
        # CASO 2
        t=t1
        C1+=1
        S.append(t)

        if len(SS) == 2:
            SS=[0]
            t1=float('inf')
        else:
            SS.pop(1)  # Remover al cliente servido
            Y1=-np.log(random.random()) / LAMBDA_2
            t1=t+Y1

    elif t2 < t1 and t2 < t_LL and len(SS) > 1:
        # CASO 3
        t=t2
        C2+=1
        S.append(t)

        if len(SS) == 2:
            SS=[0]
            t2=float('inf')
        elif len(SS) == 3:
            SS=[1, SS[1], 0]
            t2=float('inf')
        else:
            SS.pop(2)  # Remover al cliente servido
            Y2=-np.log(random.random()) / LAMBDA_2
            t2=t+Y2

print("Tiempos de llegada al sistema:", LL)
print("Tiempos de servicio en los servidores:", S)

plt.figure(figsize=(10, 6))
plt.step(LL, range(1, len(LL) + 1), where='post', label='Llegadas al sistema', color='blue')
plt.step(S, range(1, len(S) + 1), where='post', label='Servicios en los servidores', color='orange')
plt.title('Tiempos de llegada y servicio en un sistema de cola con dos servidores en paralelo')
plt.xlabel('Tiempo')
plt.ylabel('Número de eventos')
plt.legend()
plt.grid()
plt.show()
